# ReCovNet: Deep Reinforcement Learning for Solving MBCLP

## Prepare: Install dependencies
### Install with pip
* python=3.7
* PyTorch>=1.1
* numpy
* tqdm
* cv2
* tensorboard_logger

In [1]:
from utils import torch_load_cpu, load_problem, get_inner_model, move_to
from nets.attention_model import AttentionModel
from train import train_epoch, validate
from tensorboard_logger import Logger as TbLogger
from torch.utils.data import DataLoader
import numpy as np
import torch

## load the settings

In [2]:
# load the run args
%run options

# Set the random seed
torch.manual_seed(1234)

# Optionally configure tensorboard
tb_logger = None
if not opts.no_tensorboard:
    tb_logger = TbLogger(os.path.join(opts.log_dir, "{}_{}".format(opts.problem, opts.n_users, opts.n_facilities), opts.run_name))

# Set the device
use_cuda=True
opts.device = torch.device("cuda" if use_cuda else "cpu")
opts

Namespace(baseline=None, batch_size=640, bl_alpha=0.05, bl_warmup_epochs=0, checkpoint_encoder=False, checkpoint_epochs=1, data_distribution=None, device=device(type='cuda'), embedding_dim=128, epoch_size=128000, epoch_start=0, eval_batch_size=1000, eval_only=False, exp_beta=0.8, hidden_dim=128, load_path=None, log_dir='logs', log_step=50, lr_critic=0.0001, lr_decay=1, lr_model=0.0001, max_grad_norm=1.0, model='attention', n_encode_layers=3, n_epochs=500, n_facilities=100, n_users=1109, no_cuda=False, no_progress_bar=False, no_tensorboard=False, normalization='batch', output_dir='outputs', p=20, problem='MCLP', r=None, resume=None, run_name='100_20_20230906T211530', save_dir='outputs\\MCLP\\100_20_20230906T211530', seed=2023, shrink_size=None, tanh_clipping=10.0, use_cuda=True, val_dataset=None, val_size=2000)

## Figure out what's the problem

In [3]:
problem = load_problem(opts.problem)
problem

problems.MCLP.problem_MCLP.MCLP

## Initialize our policy network

In [4]:
model_class = {
    # 'pointer': PointerNetwork,
    'attention': AttentionModel
}.get(opts.model, None)

assert model_class is not None, "Unknown model: {}".format(model_class)
model = model_class(
    opts.embedding_dim,
    opts.hidden_dim,
    problem,
    n_encode_layers=opts.n_encode_layers,
    mask_inner=True,
    mask_logits=True,
    normalization=opts.normalization,
    tanh_clipping=opts.tanh_clipping,
    checkpoint_encoder=opts.checkpoint_encoder,
    shrink_size=opts.shrink_size,
    dy=False
).to(opts.device)

model

AttentionModel(
  (init_embed): Linear(in_features=2, out_features=128, bias=True)
  (init_dynamic): Linear(in_features=1, out_features=32, bias=True)
  (l2_dynamic): Linear(in_features=32, out_features=64, bias=True)
  (l3_dynamic): Linear(in_features=64, out_features=128, bias=True)
  (embedder): GraphAttentionEncoder(
    (layers): Sequential(
      (0): MultiHeadAttentionLayer(
        (0): SkipConnection(
          (module): MultiHeadAttention()
        )
        (1): Normalization(
          (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): SkipConnection(
          (module): Sequential(
            (0): Linear(in_features=128, out_features=512, bias=True)
            (1): ReLU()
            (2): Linear(in_features=512, out_features=128, bias=True)
          )
        )
        (3): Normalization(
          (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
  

## load the trained model

In [5]:
opts.load_path = './output/epoch-200.pt'
# load model from load_path
assert opts.load_path is None or opts.resume is None, "Only one of load path and resume can be given"
load_path = opts.load_path if opts.load_path is not None else opts.resume
if load_path is not None:
    print('  [*] Loading the trained model from {}'.format(load_path))
    load_data = torch_load_cpu(load_path)

# Overwrite model parameters by parameters to load q
model_ = get_inner_model(model)
model.load_state_dict({**model_.state_dict(), **load_data.get('model', {})})

  [*] Loading the trained model from ./output/epoch-200.pt


<All keys matched successfully>

In [6]:
import numpy as np
import matplotlib.pyplot as plt

## Synthetic dada n=2000, m=1000, p=15, r=0.15

In [7]:
import random
n_users = 4000
n_facilities = 2000
n_centers = 15
radius = 0.15
users = [(random.random(), random.random()) for i in range(n_users)]
facilities = [(random.random(), random.random()) for i in range(n_facilities)]
demand = np.random.randint(1, 2, size=n_users)
users, facilities = np.array(users), np.array(facilities)

In [8]:
def gen_random_data(num_sample):
    random_datasets = []
    for i in range(num_sample):
        random_data = {}
        random_data["users"] = torch.tensor(users).to(torch.float32)
        random_data["facilities"] = torch.tensor(np.array(facilities)).to(torch.float32)
        random_data['demand'] = torch.tensor(demand).to(torch.float32)
        random_data["p"] = 15
        random_data["r"] = 0.15
        random_datasets.append(random_data)
    return random_datasets

In [9]:
num_sample = 1
opts.eval_batch_size = 10
opts.max_calc_batch_size = 1280000
width = 64
real_datasets = gen_random_data(num_sample)

In [10]:
from torch.utils.data import DataLoader
from tqdm import tqdm
opts.decode_strategy = 'sampling'
model.eval()
model.set_decode_type(
    "greedy" if opts.decode_strategy in ('bs', 'greedy') else "sampling")
dataloader = DataLoader(real_datasets, batch_size=opts.eval_batch_size)


In [11]:
def get_best(sequences, cost, ids=None, batch_size=None):
    """
    Ids contains [0, 0, 0, 1, 1, 2, ..., n, n, n] if 3 solutions found for 0th instance, 2 for 1st, etc
    :param sequences:
    :param lengths:
    :param ids:
    :return: list with n sequences and list with n lengths of solutions
    """
    if ids is None:
        idx = cost.argmin()
        return sequences[idx:idx+1, ...], cost[idx:idx+1, ...]

    splits = np.hstack([0, np.where(ids[:-1] != ids[1:])[0] + 1])
    mincosts = np.minimum.reduceat(cost, splits)

    group_lengths = np.diff(np.hstack([splits, len(ids)]))
    all_argmin = np.flatnonzero(np.repeat(mincosts, group_lengths) == cost)
    result = np.full(len(group_lengths) if batch_size is None else batch_size, -1, dtype=int)

    result[ids[all_argmin[::-1]]] = all_argmin[::-1]

    return [sequences[i] if i >= 0 else None for i in result], [cost[i] if i >= 0 else math.inf for i in result]

In [12]:
start = time.time()
results = []
for batch in tqdm(dataloader, disable=True):
    batch = move_to(batch, opts.device)
    start = time.time()
    with torch.no_grad():
        if opts.decode_strategy in ('sampling', 'greedy'):
            if opts.decode_strategy == 'greedy':
                assert width == 0, "Do not set width when using greedy"
                assert opts.eval_batch_size <= opts.max_calc_batch_size, \
                    "eval_batch_size should be smaller than calc batch size"
                batch_rep = 1
                iter_rep = 1
            elif width * opts.eval_batch_size > opts.max_calc_batch_size:
                assert opts.eval_batch_size == 1
                assert width % opts.max_calc_batch_size == 0
                batch_rep = opts.max_calc_batch_size
                iter_rep = width // opts.max_calc_batch_size
            else:
                batch_rep = width
                iter_rep = 1
            assert batch_rep > 0
            # This returns (batch_size, iter_rep shape)

            sequences, costs = model.sample_many(batch, batch_rep=batch_rep, iter_rep=iter_rep)
            batch_size = len(costs)
            ids = torch.arange(batch_size, dtype=torch.int64, device=costs.device)
#         else:
#             # assert opts.decode_strategy == 'bs'

#             cum_log_p, sequences, costs, ids, batch_size = model.beam_search(
#                 batch, beam_size=width,
#                 compress_mask=opts.compress_mask,
#                 max_calc_batch_size=opts.max_calc_batch_size
#             )
            if sequences is None:
                sequences = [None] * batch_size
                costs = [math.inf] * batch_size
            else:
                sequences, costs = get_best(
                    sequences.cpu().numpy(), costs.cpu().numpy(),
                    ids.cpu().numpy() if ids is not None else None,
                    batch_size
                )
            duration = time.time() - start
            for seq, cost in zip(sequences, costs):
                seq = seq.tolist()
                results.append((cost, seq, duration))
costs, tours, durations = zip(*results)
print(f"The objective of MCBLP by DRL is: {-costs[0]}")
end = time.time()-start 
print(f"The running time of DRL is: {end}")

The objective of MCBLP by DRL is: 3672.0
The running time of DRL is: 0.5417745113372803


In [13]:
from Algorithm.GA import GeneticAlgorithm

In [14]:
dist = np.sum((facilities[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5

In [15]:
genetic = GeneticAlgorithm(n_users, n_facilities, n_centers, dist, radius, demand)
genetic.optimize()
obj = np.sum(demand) - genetic.top_chromosome.fitness
centers = genetic.top_chromosome.content

print("The Set of centers are: %s" % centers)
print("The objective is: %s" % str(round(obj)))

Current top solution: [1978, 529, 1187, 1353, 452, 154, 1556, 1098, 670, 1816, 1361, 320, 626, 1080, 605] f=1160

Final top solution: [534, 428, 1191, 1959, 648, 1677, 800, 246, 136, 1020, 1019, 1152, 1069, 504, 1668] f=611
Time: 00:00:3.4500
The Set of centers are: [534, 428, 1191, 1959, 648, 1677, 800, 246, 136, 1020, 1019, 1152, 1069, 504, 1668]
The objective is: 3389
